#FlipFeed

###Objetivo:
> Identificar cuando una persona realiza una dominada correctamente

###Metodo:
> Usaremos en este caso, redes neuronales recurrentes para obtener un valor booleano

###Herramientas:

0.   Python
1.   Keras - TensorFLow
2.   Yolo v8 - entrenado con un data set personalizado
3.   OpenCV optical flow






####Montando el directorio del drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

global dirTrV, dirOut
dirTrV = './drive/Shareddrives/flipfeed/TrainingVideos'
dirOut = './drive/Shareddrives/flipfeed/Output'

Mounted at /content/drive


##Yolo v8 y OpenCV Optical Flow
Yolov8 lo utilizamos para etiquetar los objetos en el video, podiendo así extraer las coordenadas en donde se encuentra la persona y el balón.
Acontinuación instalamos todos los paquetes necesarios.

In [ ]:
!pip install torch numpy opencv-python ultralytics supervision==0.1.0 

In [3]:
# Import the necesary libraries
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO

from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

##OpenCV Optical Flow

In [ ]:
#To draw the lines on the frame
def draw_flow(img, flow, step=20):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr

#Capture the video and generate a output with cv2
videoPath = 'dom2.mp4'
#a = open(dirTrV + videoPath, 'r')
input = cv2.VideoCapture(dirTrV + '/'+videoPath)
output = cv2.VideoWriter(dirOut + '/output_' + videoPath, cv2.VideoWriter_fourcc(*'mp4v'), 10, (int(input.get(3)),int(input.get(4))))

suc, prev = input.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

while (input.isOpened()):
    #Extract a frame from video and put on it a B&W format 
    suc, img = input.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #Obtain the motion prediction
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prevgray = gray

    #Write frames on output
    imgToShow = draw_flow(gray, flow)
    output.write(imgToShow)

output.release()
input.release()
cv2.destroyAllWindows()

##RNN implement with Keras - Tensorflow

In [11]:
import tensorflow as tf
import tensorflow.keras as kr
from tensorflow.keras import layers

lr = 0.01           # learning rate
nn = [2, 16, 8, 1]  # número de neuronas por capa.

model = kr.Sequential()

model.add(layers.Embedding(input_dim=1000, output_dim=64))
model.add(layers.SimpleRNN(128))
model.add(layers.Dense(10))

model.summary()

# Compilamos el modelo, definiendo la función de coste y el optimizador.
#model.compile(loss='mse', optimizer=kr.optimizers.SGD(lr=0.05), metrics=['acc'])

# Y entrenamos al modelo. Los callbacks 
#model.fit(X, Y, epochs=100)